In [1]:
# needed to load the REBEL model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
import math
import torch

# wrapper for wikipedia API
import wikipedia

# scraping of web articles
from newspaper import Article
# google news scraping
from GoogleNews import GoogleNews

# graph visualization
from pyvis.network import Network

# show HTML in notebook
import IPython

from keybert import KeyBERT

from sentence_transformers import SentenceTransformer
import numpy as np
import nltk
import re
from typing import List, Tuple, Union

import spacy

import inspect
import os# disabiguated_reference_liit = np.load('disambiguated_reference_list.npy')

import ollama
from tqdm import tqdm
import time
from core.SemanticSimilarity import SemanticSimilarity
from core.KG import KG
from core.KeywordExtractor import KeywordExtractor

import spacy

/home/niamatzawad/miniconda3/envs/phd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
    # from sentence_transformers import SentenceTransformer
    # from typing import List, Union, Tuple
    # import numpy as np
    
    # class SemanticSimilarity():
    #     def __init__(self):
    #         self.threshold_value = 0.6
    #         self.model = SentenceTransformer("all-MiniLM-L6-v2")
    #         self.embedded_reference = []
    #         self.disambiguated_reference_list = []
    #         self.rels = []
    #         self.embedded_rels = []
    
    #     def generate_reference(self, reference_list):
    #         temp = self._populate_embedded_reference(reference_list)
    #         x, y = self._disambiguate_reference_list_entities(reference_list, temp)
    #         self.disambiguated_reference_list, self.embedded_reference = x, y       
    
    #     def _populate_embedded_reference(self, reference_list : str) -> List[List[float]] :
    #         embedded_reference = []
    #         for i, e in enumerate(reference_list):
    #             embedded_reference.append(self._get_embedding_token(e))
    #         return embedded_reference
            
    #     def _get_embedding_token(self, phrase):
    #         return self.model.encode([phrase])[0]
            
    #     def _cosine(self, u, v) -> float:
    #         return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    
    #     def add_entity(self,e):
    #         if not self.get_most_similar_entity(e):
    #             self.disambiguated_reference_list.append(e)
    #             self.embedded_reference.append(self._get_embedding_token(e))
    
    #     def add_relation(self,r):
    #         if not self.get_most_similar_relation(r):
    #             self.rels.append(r)
    #             self.embedded_rels.append(self._get_embedding_token(r))
    
    #     def get_most_similar_entity(self, phrase):
    #         return self.get_most_similar_word(phrase, task="entity")
    
    #     def get_most_similar_entities(self, phrase):
    #         return self.get_most_similar_word(phrase, task="entity", multiple_words=True)
    
    #     def get_most_similar_relation(self, phrase):
    #         return self.get_most_similar_word(phrase, task="relation")
        
            
    #     def get_most_similar_word(self, phrase, task="entity", multiple_words=False) -> Union[None, str]:
    #         query = self._get_embedding_token(phrase)
    #         max_score = -999
    #         max_index = -1
    
    #         reference, embedded_reference = [],[]
    #         if task=="entity":
    #             reference = self.disambiguated_reference_list
    #             embedded_reference = self.embedded_reference
    #         elif task=="relation":
    #             reference = self.rels
    #             embedded_reference = self.embedded_rels
    
    #         temp_res = []
    #         for i, ref in enumerate(embedded_reference):
    #             score = self._cosine(query, ref)
    #             if score > max_score:
    #                 max_score = score
    #                 max_index = i
                
    #             temp_res.append(score)
            
    #         if not multiple_words:
    #             if max_score > self.threshold_value:
    #                 return reference[max_index]
            
    #         else:
                
    #             multiple_answers = []
    #             for i in range(len(temp_res)):
    #                 if temp_res[i] > self.threshold_value:
    #                     multiple_answers.append(reference[i])
    #             return multiple_answers
    #         return None
        
    #     def _get_all_similarity_scores(self, embedded_reference : List):
    #         return np.dot(np.array(embedded_reference), np.array(embedded_reference).T)
    
    #     def _get_similar_entities(self, exclude_index : int, l : List[float]) -> List[Tuple[float,int]]:
    #         results = []
    #         for i,e in enumerate(l):
    #             if i != exclude_index and l[i] >= self.threshold_value:
    #                 results.append((l[i],i))
    #         return results
            
    #     def _disambiguate_reference_list_entities(self, reference_list : List, embedded_reference_list : List) -> List[str]:
    #         reference_list = self.disambiguated_reference_list + reference_list
    #         embedded_reference_list = self.embedded_reference + embedded_reference_list
    #         similarity_scores = self._get_all_similarity_scores(embedded_reference_list)
    #         dissimilar,dissimilar_embedded, ignore = [],[],[]
    #         i = 0
    #         while i < len(similarity_scores):
    #             if i not in ignore:
    #                 current = similarity_scores[i]
    #                 max_score_indices = self._get_similar_entities(i, current)
    #                 ignore.extend([i for v,i in max_score_indices])
    #                 dissimilar_index = min([i] + [j for v,j in max_score_indices]) #Get the lowest index. This is because when we are parsing the second article, we want to keep any similar entities we found in the first article and remove entities any subsequent articles
    #                 dissimilar.append(reference_list[dissimilar_index])
    #                 dissimilar_embedded.append(embedded_reference_list[dissimilar_index])
    #                 # dissimilar.append(max([reference_list[i]] + [reference_list[j] for v,j in max_score_indices])) #assuming larger text carries more information
    #             i += 1
    
    #         return dissimilar, dissimilar_embedded


In [3]:
disambiguated_reference_list = np.load('disambiguated_reference_list_kg_demo.npy')
embedded_reference = np.load('embedded_reference_kg_demo.npy')
xs = SemanticSimilarity()
xs.embedded_reference = embedded_reference
xs.disambiguated_reference_list = disambiguated_reference_list

In [4]:
xs.get_most_similar_entities("administrative_judge")

['administrative judge', 'administrators']

In [5]:
from py2neo import Graph, Node, Relationship



# Connect to the Neo4j database
graph = Graph("neo4j://localhost:7687", auth=("neo4j", "password"))

In [6]:
xs.get_most_similar_entities("Texas")

['missouri', 'louisiana', 'kentucky state', 'oklahoma', 'alaska', 'austin tex']

In [7]:
kwe = KeywordExtractor()

In [8]:
kwe.extract_named_entities("What did Mr McGovern say regarding right-to-lifers?")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['mr mcgovern']

In [19]:
class Search:
    def __init__(self, s : SemanticSimilarity, kwe : KeywordExtractor, graph : Graph):
        self.s = s
        self.kwe = kwe
        self.nlp = spacy.load("en_core_web_sm")


    def _retrieve_keywords(self, question):
        return self.kwe.extract_keywords(question)

    def _retrieve_verbs(self, question):
        doc = self.nlp(question)
        return [token.lemma_ for token in doc if token.pos_ == "VERB"]
        
    def _get_relevant_texts(self, keywords):
        all_texts = []
        for k in keywords:
            graph_entities = self.s.get_most_similar_entities(k)
            print(f"the keywords extracted are {keywords}")
            print('the most similar entitiesb are ')
            print(graph_entities)
            for graph_entity in graph_entities:
                
                graph_entity = re.sub(" ", "_", graph_entity)
                d = graph.run(f"MATCH (n:{graph_entity}) RETURN n").data()
                if d:
                    nodes = graph.run(f"MATCH p = (n:{graph_entity})-[r]->(m) WHERE type(r) <> 'filepath' RETURN p,r['text'] as text, type(r) as relationType ").data()
                    for n in nodes:
                        print(n[])
                        print(n['relationType'])
                        all_texts.append(n["text"])

                    # text = graph.run(f"MATCH p = (n:{graph_entity})-[r:filepath]->(m) RETURN r['text'] as text ").data()
        return all_texts

    def get_answers(self, question):
        keywords = self._retrieve_keywords(question)
        all_texts = self._get_relevant_texts(keywords)
        all_texts = [i for i in all_texts if i]
        question_verb_phrases = self._retrieve_verbs(question)
        print(all_texts)
        all_texts =  "\n".join(all_texts)
        print("Texts")
        print(all_texts)
        return ollama.generate('mistral', prompt=f"Given the question: {question}, generate an answer from the following text. \n ###The text: {all_texts}")['response']
        

    



    

In [20]:
xs.get_most_similar_entity('president carter')

'president carter'

In [21]:
xs.get_most_similar_entities('mr mcgovern')

['mr gov']

In [22]:
search_obj = Search(xs, kwe, graph)
search_obj.get_answers('Who is Frank S. Kristof?')

the keywords extracted are ['kristof', 'frank']
the most similar entities are 
[]
the keywords extracted are ['kristof', 'frank']
the most similar entities are 
['frank']
[]
Texts



' Frank S. Kristof is not explicitly mentioned in the provided text. Therefore, I cannot generate an answer based on the given text alone that directly addresses who Frank S. Kristof is. If there is additional context or information about Frank S. Kristof available, please provide it so I can help answer your question.'

In [23]:
search_obj.get_answers('What did President Carter say?')

the keywords extracted are ['carter', 'president', 'say']
the most similar entities are 
['president carter', 'carter b hors ley']
(_2714)-[:hope_for {text: 'Mr. Aiello praised Polish-Americans for their economic and political contributions to the United States and said he hoped the crisis in Poland "would be resolved within the context of the human rights accords signed at Helsinki."'}]->(_3591)
(_2714)-[:received {text: 'In an apparent attempt to offset some of the disappointment and criticism among Polish-Americans, Mr. Carter sent Stephen Aiello, his special assistant for ethnic affairs, to a rally in Chicago yesterday.'}]->(_3588)
(_2714)-[:revived {text: 'President Carter revived draft registration, but not the draft itself, after the Soviet military intervention in Afghanistan in December 1979.'}]->(_3425)
(_2714)-[:addressed {text: 'In one of the incidents, 15 policemen were injured outside Madison Square Garden just before President Carter addressed the Democratic National Con

" At a rally in Chicago, President Carter's special assistant for ethnic affairs, Stephen Aiello, praised Polish-Americans for their economic and political contributions to the United States and expressed hope that the crisis in Poland would be resolved within the context of the human rights accords signed at Helsinki. The President had previously sent Aiello to the rally as an attempt to address disappointment and criticism among Polish-Americans. In response to the Soviet military intervention in Afghanistan, Carter revived draft registration but not the draft itself. During his campaign in Dallas, Carter once again defended this move as a precautionary measure."

In [32]:
question = "What did the administrative judge do?"
keywords = kwe.extract_named_entities(question)
for k in keywords:
    graph_entity = s.get_most_similar_entity(k)
    graph_entity = re.sub(" ", "_", graph_entity)
    d = graph.run(f"MATCH (n:{graph_entity}) RETURN n").data()
    if d:
        nodes = graph.run(f"MATCH p = (n:{graph_entity})-[r]->(m) WHERE type(r) <> 'filepath' RETURN p,r['text'] as text ").data()
        full_text = ""
        for n in nodes:
            print(n["text"])
        print("---")
        print("the full text:")
        text = graph.run(f"MATCH p = (n:{graph_entity})-[r:filepath]->(m) RETURN r['text'] as text ").data()
        print(len(text))                                                                                                                                                                                                                                                                                                                              




NameError: name 's' is not defined

In [162]:

d = graph.run("MATCH (p:allan_cunningham)-[q]->(r) RETURN p,q['text'] as text, r").data()
if d:
    for a in d:
        print(a["text"])

Dr. Allan S. Cunningham said that studies at The Mary Imogene Bassett Hospital had shown that bottle-fed babies were hospitalized three times more frequently than breast-fed infants in the first year of life and 15 times more frequently in the first 15 months.
Dr. Allan S. Cunningham said that studies at The Mary Imogene Bassett Hospital had shown that bottle-fed babies were hospitalized three times more frequently than breast-fed infants in the first year of life and 15 times more frequently in the first 15 months.


In [33]:
"filepath" in d[0]["p"]

False

In [27]:
for en in d:
    print(en)
    if "filepath" in en:
        print(en["text"])
        break

{'p': Path(Node('dave_rogers', id='dave_rogers'), expresses_opinion(Node('dave_rogers', id='dave_rogers'), Node('catholic', id='catholic')))}
{'p': Path(Node('dave_rogers', id='dave_rogers'), filepath(Node('dave_rogers', id='dave_rogers'), Node('`/home/niamatzawad/niamatzawad/Datasets/UTDBox/NYT Files/1-1000/NYT-8006042-8006063.TXT`', text="\ufeff\n                                1 of 8 DOCUMENTS\n\n                               The New York Times\n\n                 June 22, 1980, Sunday, Late City Final Edition\n\nSTRIKE AND MOON GROUP'S ARRIVAL UPSET KODIAK FISHING INDUSTRY\n\nBYLINE: By WALLACE TURNER, Special to the New York Times\n\nSECTION: Section 1; Part 1; Page 24, Column 1; National Desk\n\nLENGTH: 1000 words\n\nDATELINE: KODIAK, Alaska, June 18\n\n\nThe sun glitters on a harbor full of fishing boats where nets are being patched,\nvarnish is being sanded and running gear repaired, and this fishing port appears\npurposeful and contented.\n\nIt is not. The place is tense. The

In [149]:
graph.run(f"MATCH (n:{s.get_most_similar_entity("Alfred B. DelBello")}) RETURN n;").data()

ServiceUnavailable: Cannot connect to any known routers

In [75]:
graph.run(f"MATCH (n:roger) RETURN n;").data()

[{'n': Node('roger', id='roger')}]

In [2]:
kwe = KeywordExtractor()

In [22]:
kwe.extract_named_entities("who is Mr. Mindel?")

['mr mind el']

In [ ]:
kwe.extract_named_entities("")

In [146]:
kwe

NameError: name 'kwe' is not defined